# Introduction

L'objectif de ce notebook est d'appliquer des techniques de ré-échantillonnage pour rééquilibrer les classes fournies dans le dataset original.

Son rôle est également de fournir une base de données commune qui pourra être utilisée par les différents notebooks de modélisations.

# Initialisation

In [1]:
import sys
import os
from pathlib import Path

import pandas as pd

from sklearn.model_selection import train_test_split

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [2]:
random_state = 42

In [3]:

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    %cd drive/MyDrive/nov23_bds_rakuten
else:
    root_files = os.listdir()
    if not "requirements.txt" in root_files:
        print("The jupyter server root should be set to root of git project")
        print("If you're on vscode add the following to your settings.json file")
        print('"jupyter.notebookFileRoot": "${workspaceFolder}"')

Mounted at /content/drive/
/content/drive/MyDrive/nov23_bds_rakuten


# Import des données

Importons les données de sorties du notebook **data_preprocessing_images**

In [4]:
df = pd.read_csv('output/data-preprocessing-images/df_pre.csv', index_col=0)
df.head()

,productid,imageid,prdtypecode,text,lang,text_fr,imagefile
index,,,,,,,
0,3804725264,1263597046,10,Olivia: Personalisiertes Notizbuch / 150 Seite...,de,Olivia: cahier personnalisé / 150 pages / gril...,10/image_1263597046_product_3804725264.jpg
1,436067568,1008141237,2280,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,fr,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,2280/image_1008141237_product_436067568.jpg
2,201115110,938777978,50,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,fr,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,50/image_938777978_product_201115110.jpg
3,50418756,457047496,1280,Peluche Donald - Europe - Disneyland 2000 (Mar...,de,Peluche Donald - Europe - Disneyland 2000 (Mar...,1280/image_457047496_product_50418756.jpg
4,278535884,1077757786,2705,La Guerre Des Tuques - Luc a des idées de gran...,fr,La Guerre Des Tuques - Luc a des idées de gran...,2705/image_1077757786_product_278535884.jpg


Créons deux variables **X** et **y** qui vont contenir respectivement, les variables explicatives et la variable cible.

On choisi de conserver dans **X** quelques variables qui ne sont pas des features mais qui pourront néanmoins nous aider lors de l'analyse des résultats de nos modèles

In [5]:
X = df.drop("prdtypecode",axis=1)
y = df["prdtypecode"]

# Séparation en 3 sets: train, validation et test

Commençons par scinder les données en un set d’entraînement (**X_train**,**y_train**) et un set de test (**X_test**,**y_test**).

Ce premier ensemble de test comprend 20% des données choisies aléatoirement mais en respectant la répartition initiale des classes. On veille également à toujours passer le même **random_state** pour s'assurer que la fonction renvoie toujours le même résultat si on doit ré-executer ce notebook plus tard.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = random_state,shuffle=True,stratify=y)

On va maintenant couper en deux l'ensemble de test pour que la moitié de ses données aillent dans un ensemble de validation (**X_val**,**y_val**).



In [7]:
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size=0.5,random_state = random_state,shuffle=True,stratify=y_test)

On se retrouve donc avec 3 ensembles des données:
* Un ensemble d’entraînement composé de 80% des données
* Un ensemble de validation composé de 10% des données
* Un ensemble de test composé de 10% des données

# Equilibrage des classes

On a pu constater dans le notebook **data-exploration** que les classes ne sont pas équilibrées. On va appliquer des techniques de ré-échantillonnage pour tenter de compenser ces inégalités au mieux.

Pour rappel, voici le nombre d’occurrences de chaque classe

In [8]:
display(y_train.value_counts())

2583    8167
1560    4058
1300    4036
2060    3994
2522    3991
1280    3896
2403    3819
2280    3808
1920    3442
1160    3162
1320    2593
10      2493
2705    2209
1140    2137
2582    2071
40      2006
2585    1997
1302    1993
1281    1656
50      1345
2462    1137
2905     698
60       666
2220     659
1301     646
1940     642
1180     611
Name: prdtypecode, dtype: int64

On commencer par réduire le nombre d'observations dans la classe **2583** car celle-ci contient beaucoup plus de données que les autres.

In [9]:
under_strategy  = {2583: 4058}
# Appliquer le sous-échantillonnage
under_sampler = RandomUnderSampler(sampling_strategy = under_strategy,random_state=random_state)
X_train, y_train = under_sampler.fit_resample(X_train, y_train)

On va maintenant augmenter le nombre d’occurrences des classes minoritaires pour qu'il atteigne environ **4058**, ce nombre représentant la population des classes moyennement peuplées du dataset

In [11]:
ros = RandomOverSampler(random_state=random_state)
X_train, y_train = ros.fit_resample(X_train, y_train)

Affichons une nouvelle fois le nombre d’occurrence par classe pour constater le résultat de nos opérations

In [12]:
display(y_train.value_counts())

10      4058
1920    4058
2705    4058
2585    4058
2583    4058
2582    4058
2522    4058
2462    4058
2403    4058
2280    4058
2220    4058
2060    4058
1940    4058
1560    4058
40      4058
1320    4058
1302    4058
1301    4058
1300    4058
1281    4058
1280    4058
1180    4058
1160    4058
1140    4058
60      4058
50      4058
2905    4058
Name: prdtypecode, dtype: int64

On constate que les classes sont maintenant rigoureusement équilibrées

# Export des données

Exportons les trois ensembles de données dans le dossier output du notebook pour qu'ils soient accessibles par les notebooks de modélisations.

In [13]:
output_dir = "output/data-preprocessing-resampling"
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [14]:
df_train = pd.concat([X_train,y_train],axis=1)
display(df_train.head())
display(df_train.info())
df_train.to_csv(output_dir+"/df_train.csv")


,productid,imageid,text,lang,text_fr,imagefile,prdtypecode
0,131641431,885888766,Jesus - Cahiers Du Libre Avenir - Prêtre autre...,fr,Jesus - Cahiers Du Libre Avenir - Prêtre autre...,10/image_885888766_product_131641431.jpg,10
1,745789761,1304710852,The Inglorious Arts Of Peace: Exhibitions In C...,en,Les arts de la paix sans gloire: expositions d...,10/image_1304710852_product_745789761.jpg,10
2,651353882,1033554185,Lecons De Sciences - Cours Superieur - Premier...,fr,Lecons De Sciences - Cours Superieur - Premier...,10/image_1033554185_product_651353882.jpg,10
3,4043473264,1306457352,Generations Of Women Historians -,en,Générations de femmes historiens -,10/image_1306457352_product_4043473264.jpg,10
4,679913105,1034656535,Oeuvres Dramatiques De F. Schiller Tome 6 -,fr,Oeuvres Dramatiques De F. Schiller Tome 6 -,10/image_1034656535_product_679913105.jpg,10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109566 entries, 0 to 109565
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   productid    109566 non-null  int64 
 1   imageid      109566 non-null  int64 
 2   text         109566 non-null  object
 3   lang         109566 non-null  object
 4   text_fr      109566 non-null  object
 5   imagefile    109566 non-null  object
 6   prdtypecode  109566 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 5.9+ MB


None

In [15]:
df_val = pd.concat([X_val,y_val],axis=1)
display(df_val.head())
display(df_val.info())
df_val.to_csv(output_dir+"/df_val.csv")


,productid,imageid,text,lang,text_fr,imagefile,prdtypecode
index,,,,,,,
64448,1951685748,1317996370,Linxor - Bâche à Bulles 300 microns Piscine 3m...,fr,Linxor - Bâche à Bulles 300 microns Piscine 3m...,2583/image_1317996370_product_1951685748.jpg,2583
37973,159494381,1029666610,Kit piscine autoportante Easy Set Ø 305 x 076m -,fr,Kit piscine autoportante Easy Set Ø 305 x 076m -,2583/image_1029666610_product_159494381.jpg,2583
73967,4011564183,1276163958,Kayak Gonflable Betta Hm K0 Biplace Et Ses Acc...,fr,Kayak Gonflable Betta Hm K0 Biplace Et Ses Acc...,1302/image_1276163958_product_4011564183.jpg,1302
33180,3797074222,1246071438,Mini Coloré En Plastique Pot De Fleurs Succule...,fr,Mini Coloré En Plastique Pot De Fleurs Succule...,2582/image_1246071438_product_3797074222.jpg,2582
26659,4157458648,1303079928,Modèle De Voiturerc Buggy Truck Chassis Body L...,fr,Modèle De Voiturerc Buggy Truck Chassis Body L...,1300/image_1303079928_product_4157458648.jpg,1300


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8492 entries, 64448 to 34039
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   productid    8492 non-null   int64 
 1   imageid      8492 non-null   int64 
 2   text         8492 non-null   object
 3   lang         8492 non-null   object
 4   text_fr      8492 non-null   object
 5   imagefile    8492 non-null   object
 6   prdtypecode  8492 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 530.8+ KB


None

In [16]:
df_test = pd.concat([X_test,y_test],axis=1)
display(df_test.head())
display(df_test.info())
df_test.to_csv(output_dir+"/df_test.csv")


,productid,imageid,text,lang,text_fr,imagefile,prdtypecode
index,,,,,,,
29927,3469563513,1215209457,Adventure Parks Heavy Duty Tree Strap - Dont t...,en,Les parcs d'aventure STRAP DURT DE DURG - N'at...,1280/image_1215209457_product_3469563513.jpg,1280
42596,4152889175,1309607798,Magideal 2 Pcs Orff Instrument De Musique Hoch...,fr,Magideal 2 Pcs Orff Instrument De Musique Hoch...,1320/image_1309607798_product_4152889175.jpg,1320
158,1190507078,1068211845,Casa Padrino Baroque Dining Room With Armrests...,en,Casa Padrino Baroque Dining Room avec des acco...,1560/image_1068211845_product_1190507078.jpg,1560
20061,4236041996,1324088536,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,fr,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,1302/image_1324088536_product_4236041996.jpg,1302
20931,3836739131,1250952042,Organisateur Enfants En Bois Keepsake Cadeau P...,fr,Organisateur Enfants En Bois Keepsake Cadeau P...,1280/image_1250952042_product_3836739131.jpg,1280


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8492 entries, 29927 to 18545
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   productid    8492 non-null   int64 
 1   imageid      8492 non-null   int64 
 2   text         8492 non-null   object
 3   lang         8492 non-null   object
 4   text_fr      8492 non-null   object
 5   imagefile    8492 non-null   object
 6   prdtypecode  8492 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 530.8+ KB


None